In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import bokeh.plotting as bpl
import bokeh.palettes as bpt
import colorcet as cc
import json

In [2]:
bpl.output_notebook()

Loading BokehJS ...

Para ejecutar estos análisis primero se tiene que ejecutar el notebook del archivo "datos_historicos.ipynb", ese notebook permite descargar los archivos de datos con los que trabaja este notebook.

In [3]:
primera_fecha = "04/12/2020" #No hay datos antes del 04/12/2020 en las bases federales
ultima_fecha = "06/30/2020"

In [4]:
fechas = pd.date_range(start=primera_fecha, end=ultima_fecha)

Carga de datos históricos y actuales y conversión a DataFrame, para cada archivo se agrega una columna al dataframe que hace referencia a la fecha de la publicación del archivo, el país va a estar roto por problemas de encoding. En libre office se pueden abrir como utf8 pero python detecta un caracter que no puede decodificar y no abre los archivos. Se están forzando a 'latin1' pero entonces los acentos no son correctos.

In [5]:
fechas_csv = fechas.strftime("%y%m%d")+"COVID19MEXICO.csv"

In [6]:
frames = []
for fecha in fechas_csv:
    data = pd.read_csv('./datos_federales/' + fecha,encoding="latin1", low_memory=False)
    data.loc[:,"FECHA_ARCHIVO"] = dt.datetime.strptime(fecha[:6],"%y%m%d").date()
    frames.append(data)

In [7]:
len(frames)

80

In [22]:
all_data = pd.concat(frames,axis=0,ignore_index=True)

Conversión de fechas a formato de fecha

In [23]:
#all_data["FECHA_ARCHIVO"] = pd.to_datetime(all_data["FECHA_ARCHIVO"],format="%Y-%m-%d")
#all_data["FECHA_ARCHIVO_c"] = all_data["FECHA_ARCHIVO"].dt.strftime(date_format="%d-%m")
#all_data["FECHA_ACTUALIZACION"] = pd.to_datetime(all_data["FECHA_ACTUALIZACION"],format="%Y-%m-%d")
#all_data["FECHA_ACTUALIZACION_c"] = all_data["FECHA_ACTUALIZACION"].dt.strftime(date_format="%d-%m")
#all_data["FECHA_INGRESO"] = pd.to_datetime(all_data["FECHA_INGRESO"],format="%Y-%m-%d")
#all_data["FECHA_INGRESO_c"] = all_data["FECHA_INGRESO"].dt.strftime(date_format="%d-%m")
#all_data["FECHA_DEF"] = pd.to_datetime(all_data["FECHA_DEF"].replace({"9999-99-99":None}),format="%Y-%m-%d")
#all_data["FECHA_DEF_c"] = all_data["FECHA_DEF"].dt.strftime(date_format="%d-%m")
#all_data["FECHA_SINTOMAS"] = pd.to_datetime(all_data["FECHA_SINTOMAS"],format="%Y-%m-%d")
#all_data["FECHA_SINTOMAS_c"] = all_data["FECHA_SINTOMAS"].dt.strftime(date_format="%d-%m")

In [24]:
all_data["FECHA_ARCHIVO"] = pd.to_datetime(all_data["FECHA_ARCHIVO"],format="%Y-%m-%d")
all_data["FECHA_ACTUALIZACION"] = pd.to_datetime(all_data["FECHA_ACTUALIZACION"],format="%Y-%m-%d")
all_data["FECHA_INGRESO"] = pd.to_datetime(all_data["FECHA_INGRESO"],format="%Y-%m-%d")
all_data["FECHA_DEF"] = pd.to_datetime(all_data["FECHA_DEF"].replace({"9999-99-99":None}),format="%Y-%m-%d")
all_data["FECHA_SINTOMAS"] = pd.to_datetime(all_data["FECHA_SINTOMAS"],format="%Y-%m-%d")
all_data["ENTIDAD_UM"] = all_data["ENTIDAD_UM"].astype(str).str.zfill(2)
all_data["ENTIDAD_NAC"] = all_data["ENTIDAD_NAC"].astype(str).str.zfill(2)
all_data["ENTIDAD_RES"] = all_data["ENTIDAD_RES"].astype(str).str.zfill(2)
all_data["MUNICIPIO_RES"] = all_data["MUNICIPIO_RES"].fillna(0).astype(int).astype(str).str.zfill(3)
all_data["CVEGEO_RES"] = all_data["ENTIDAD_RES"] + all_data["MUNICIPIO_RES"]

In [25]:
all_data.columns

Index(['FECHA_ACTUALIZACION', 'ID_REGISTRO', 'ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES', 'TIPO_PACIENTE',
       'FECHA_INGRESO', 'FECHA_SINTOMAS', 'FECHA_DEF', 'INTUBADO', 'NEUMONIA',
       'EDAD', 'NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'DIABETES',
       'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'RESULTADO', 'MIGRANTE', 'PAIS_NACIONALIDAD',
       'PAIS_ORIGEN', 'UCI', 'FECHA_ARCHIVO', 'CVEGEO_RES'],
      dtype='object')

In [90]:
confirmados = all_data[all_data["RESULTADO"]==1]
difuntos = confirmados[confirmados["FECHA_DEF"].notna()]

In [91]:
pacientes_mun = confirmados.groupby(["ID_REGISTRO","CVEGEO_RES","ENTIDAD_RES","MUNICIPIO_RES"]).size().reset_index().drop(columns=[0])
pacientes_edo = confirmados.groupby(["ID_REGISTRO","ENTIDAD_RES"]).size().reset_index().drop(columns=[0])

In [92]:
varios_mun = pacientes["ID_REGISTRO"].value_counts()[pacientes["ID_REGISTRO"].value_counts()>1].index
varios_edo = pacientesedo["ID_REGISTRO"].value_counts()[pacientesedo["ID_REGISTRO"].value_counts()>1].index

In [89]:
confirmados[:3]

,FECHA_ACTUALIZACION,ID_REGISTRO,ORIGEN,SECTOR,ENTIDAD_UM,SEXO,ENTIDAD_NAC,ENTIDAD_RES,MUNICIPIO_RES,TIPO_PACIENTE,...,RENAL_CRONICA,TABAQUISMO,OTRO_CASO,RESULTADO,MIGRANTE,PAIS_NACIONALIDAD,PAIS_ORIGEN,UCI,FECHA_ARCHIVO,CVEGEO_RES
0,2020-04-19,09e8dc,2,9,15,1,15,15,037,2,...,2,2,2,1,99,99,97,1,2020-04-12,15037
1,2020-04-19,0efbaf,2,9,28,2,16,28,032,1,...,2,2,1,1,99,99,97,97,2020-04-12,28032
4,2020-04-19,0cdb12,1,12,09,2,99,09,012,2,...,2,2,2,1,99,MÃ©xico,97,2,2020-04-12,09012


In [136]:
def geomatriz(frame,fecha_sel,pacientes_edo,varios_edo,pacientes_mun,varios_mun):
    por_fecha = frame.groupby(["ID_REGISTRO",fecha_sel]).size().reset_index().pivot(index=fecha_sel,columns="ID_REGISTRO",values=0).sort_index().T
    fechas_validas = por_fecha.columns
    validos_mun = pd.merge(pacientes_mun,por_fecha.reset_index()[~por_fecha.reset_index()["ID_REGISTRO"].isin(varios_mun)],how="right",on="ID_REGISTRO")
    validos_edo = pd.merge(pacientes_edo,por_fecha.reset_index()[~por_fecha.reset_index()["ID_REGISTRO"].isin(varios_edo)],how="right",on="ID_REGISTRO")
    validos_edo["ENTIDAD_RES"] = validos_edo["ENTIDAD_RES"].str.zfill(5)
    edo_frame = validos_edo.groupby("ENTIDAD_RES")[fechas_validas].sum().T
    mun_frame = validos_mun.groupby("CVEGEO_RES")[fechas_validas].sum().T
    nac_frame = por_fecha.sum().to_frame("00000")
    return pd.concat([nac_frame,edo_frame,mun_frame],axis=1)

In [137]:
casos = geomatriz(confirmados,"FECHA_ARCHIVO",pacientes_edo,varios_edo,pacientes_mun,varios_mun)

In [138]:
casos[-3:]

,00000,00001,00002,00003,00004,00005,00006,00007,00008,00009,...,32046,32048,32049,32051,32052,32053,32054,32055,32056,32057
2020-06-28,216852.0,2219.0,8720.0,1437.0,1810.0,3999.0,526.0,4337.0,2956.0,47032.0,...,7.0,0.0,6.0,13.0,8.0,2.0,2.0,5.0,161.0,6.0
2020-06-29,220657.0,2257.0,8753.0,1487.0,1859.0,4270.0,539.0,4381.0,3005.0,47422.0,...,8.0,2.0,7.0,13.0,8.0,2.0,2.0,6.0,169.0,6.0
2020-06-30,226089.0,2340.0,8921.0,1529.0,1916.0,4573.0,548.0,4487.0,3060.0,47999.0,...,8.0,3.0,7.0,13.0,8.0,2.0,2.0,6.0,170.0,6.0


In [154]:
casos.to_csv("./datos_procesados/casos.csv")

In [155]:
defunciones = geomatriz(difuntos,"FECHA_ARCHIVO",pacientes_edo,varios_edo,pacientes_mun,varios_mun)

In [156]:
defunciones[-3:]

,00000,00001,00002,00003,00004,00005,00006,00007,00008,00009,...,32040,32042,32045,32046,32049,32051,32052,32054,32055,32056
2020-06-28,26648.0,128.0,1832.0,70.0,179.0,211.0,59.0,524.0,627.0,5275.0,...,3.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,1.0,11.0
2020-06-29,27121.0,134.0,1848.0,70.0,182.0,215.0,62.0,541.0,635.0,5339.0,...,3.0,2.0,2.0,3.0,2.0,2.0,2.0,2.0,1.0,12.0
2020-06-30,27769.0,140.0,1878.0,71.0,191.0,231.0,64.0,567.0,640.0,5413.0,...,3.0,2.0,2.0,3.0,3.0,2.0,2.0,2.0,1.0,12.0


In [157]:
defunciones.to_csv("./datos_procesados/defunciones.csv")

In [158]:
fechas = confirmados["FECHA_ARCHIVO"].sort_values().unique()

In [185]:
for fecha in fechas:
    fecha_pan = pd.to_datetime(fecha)
    filtrados = confirmados[confirmados["FECHA_ARCHIVO"]==fecha_pan]
    matriz_filt = geomatriz(filtrados,"FECHA_SINTOMAS",pacientes_edo,varios_edo,pacientes_mun,varios_mun)
    matriz_filt.cumsum().to_csv("./datos_procesados/casos_sint_" + fecha_pan.strftime("%d-%m") + ".csv")

In [186]:
for fecha in fechas:
    fecha_pan = pd.to_datetime(fecha)
    filtrados = difuntos[difuntos["FECHA_ARCHIVO"]==fecha_pan]
    matriz_filt = geomatriz(filtrados,"FECHA_SINTOMAS",pacientes_edo,varios_edo,pacientes_mun,varios_mun)
    matriz_filt.cumsum().to_csv("./datos_procesados/defun_sint_" + fecha_pan.strftime("%d-%m") + ".csv")